In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [2]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [3]:
spark

In [4]:
socketDF = spark.readStream.format("socket").option("host", "seppe.net").option("port", 7778).load()

socketDF.printSchema()

root
 |-- value: string (nullable = true)



In [5]:
from pyspark.sql.functions import from_json, schema_of_json

In [6]:
def process_row(df, epoch_id):
    print(epoch_id)
    if df.count() == 0: return
    schema = schema_of_json(df.first().value)
    df_cols = df.selectExpr('CAST(value AS STRING)')\
        .select(from_json('value', schema)\
        .alias('temp'))\
        .select('temp.*')
    df_cols.show()

In [7]:
query = socketDF.writeStream.trigger(processingTime='5 seconds').foreachBatch(process_row).start()  

0
1
+------+-------------------+--------------------+
| label|           tweet_id|          tweet_text|
+------+-------------------+--------------------+
|#china|1379371830071418880|🇨🇳 #███████ ‘co...|
+------+-------------------+--------------------+

2
+------+-------------------+--------------------+
| label|           tweet_id|          tweet_text|
+------+-------------------+--------------------+
|#china|1379371809917788162|Surviving the Cra...|
+------+-------------------+--------------------+

3
+------+-------------------+--------------------+
| label|           tweet_id|          tweet_text|
+------+-------------------+--------------------+
|#china|1379371804054056964|#███████ From #██...|
+------+-------------------+--------------------+

4
+------+-------------------+--------------------+
| label|           tweet_id|          tweet_text|
+------+-------------------+--------------------+
|#china|1379371734244204550|Global #███████ a...|
+------+-------------------+-----------

In [8]:
query.stop()